In [24]:
from sklearn.cluster import DBSCAN
import pandas as pd

In [10]:
from sklearn.preprocessing import StandardScaler
from trackml.dataset import load_event
import pandas as pd
import numpy as np
import glob
import scipy as sp

train = np.unique([p.split('-')[0] for p in sorted(glob.glob('C:/Users/sumas/Downloads/train_sample/train_100_events/**'))])
test = np.unique([p.split('-')[0] for p in sorted(glob.glob('C:/Users/sumas/Downloads/test/test/**'))])
data = pd.read_csv('C:/Users/sumas/Downloads/train_sample/train_100_events/event000001000-hits.csv')
sub = pd.read_csv('C:/Users/sumas/Downloads/sample_submission.csv/sample_submission.csv')
print(len(train), len(test), len(data), len(sub))
data.head()

100 125 120939 13741466


,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.4099,-7.163700,-1502.5,7,2,1
1,2,-55.3361,0.635342,-1502.5,7,2,1
2,3,-83.8305,-1.143010,-1502.5,7,2,1
3,4,-96.1091,-8.241030,-1502.5,7,2,1
4,5,-62.6736,-9.371200,-1502.5,7,2,1


In [4]:
x = data.ix[:,0:5].values
y = data.ix[:,5].values

#Visualizing
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

C:\Users\sumas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [5]:
#standardizing
from sklearn.preprocessing import StandardScaler
x_std = StandardScaler().fit_transform(x)
x_std

array([[-1.73203649, -0.20489894, -0.03904272, -1.4127258 , -1.1651586 ],
       [-1.73200784, -0.17521825, -0.01348649, -1.4127258 , -1.1651586 ],
       [-1.7319792 , -0.26842436, -0.01931387, -1.4127258 , -1.1651586 ],
       ...,
       [ 1.7319792 , -3.20942782,  0.12029243,  2.78255264,  2.10311532],
       [ 1.73200784, -3.07781128,  0.04501746,  2.78255264,  2.10311532],
       [ 1.73203649, -3.01301539, -0.00871047,  2.78255264,  2.10311532]])

In [6]:
#Covarience Matrix
mean_vec = np.mean(x_std, axis=0)
cov_mat = (x_std - mean_vec).T.dot((x_std - mean_vec)) / (x_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)

Covariance matrix 
[[ 1.00000827 -0.00216942  0.01817127  0.31615765  0.9561355 ]
 [-0.00216942  1.00000827  0.00288111  0.00605698 -0.00364237]
 [ 0.01817127  0.00288111  1.00000827 -0.001488    0.01698981]
 [ 0.31615765  0.00605698 -0.001488    1.00000827  0.16617502]
 [ 0.9561355  -0.00364237  0.01698981  0.16617502  1.00000827]]


In [7]:
cov_mat = np.cov(x_std.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[ 6.84279148e-01 -7.16730065e-01 -1.34110355e-01  7.10531036e-03
   4.89776298e-03]
 [-1.85534534e-03  2.50613399e-04 -7.41961234e-02 -7.23054405e-01
  -6.86792368e-01]
 [ 2.17692926e-02  1.56211745e-03  9.23240978e-02 -6.90578687e-01
   7.17008365e-01]
 [ 3.05934647e-01  1.15983277e-01  9.37610715e-01  1.39854844e-02
  -1.16800794e-01]
 [ 6.61580452e-01  6.87636088e-01 -2.98113664e-01  6.87933556e-03
   2.34271894e-02]]

Eigenvalues 
[2.06636216 0.03148448 0.90132569 1.00269879 0.99817022]


In [8]:
#Correlation MAtrix
cor_mat1 = np.corrcoef(x_std.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat1)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[ 6.84279148e-01 -7.16730065e-01 -1.34110355e-01  7.10531036e-03
   4.89776298e-03]
 [-1.85534534e-03  2.50613399e-04 -7.41961234e-02 -7.23054405e-01
  -6.86792368e-01]
 [ 2.17692926e-02  1.56211745e-03  9.23240978e-02 -6.90578687e-01
   7.17008365e-01]
 [ 3.05934647e-01  1.15983277e-01  9.37610715e-01  1.39854844e-02
  -1.16800794e-01]
 [ 6.61580452e-01  6.87636088e-01 -2.98113664e-01  6.87933556e-03
   2.34271894e-02]]

Eigenvalues 
[2.06634508 0.03148422 0.90131824 1.0026905  0.99816196]


In [19]:
#SVD
u,s,v = sp.sparse.linalg.svds(x_std.T, k=4)
u

array([[-0.13411036,  0.00489776,  0.00710531,  0.68427915],
       [-0.07419612, -0.68679237, -0.72305441, -0.00185535],
       [ 0.0923241 ,  0.71700837, -0.69057869,  0.02176929],
       [ 0.93761071, -0.11680079,  0.01398548,  0.30593465],
       [-0.29811366,  0.02342719,  0.00687934,  0.66158045]])

In [20]:
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Everything ok!')

Everything ok!


In [21]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

Eigenvalues in descending order:
2.0663450773245557
1.0026904966748182
0.998161961968904
0.9013182397408348
0.03148422429089148


In [25]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
py.iplot(fig)

Aw, snap! We didn't get a username with your request.

Don't have an account? https://plot.ly/api_signup

Questions? accounts@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.